In [1]:
import pymysql,sys,json
import seaborn as sns
import traceback
import cPickle as pickle
import collections
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn.decomposition import PCA
import sklearn

In [2]:
conn = pymysql.connect(host='localhost',user='root',passwd='pulsetini', db='nairaland',charset='utf8')
cur = conn.cursor()
cur2 = conn.cursor()

###Get usernames and genders

In [3]:
%time usersDf=pd.read_sql_query('select userName,gender,posts from users',conn)

CPU times: user 2.69 s, sys: 50.8 ms, total: 2.74 s
Wall time: 2.87 s


###Get posts: users and topics

In [4]:
%time postsDf=pd.read_sql_query('select user,topic,count(*) from posts3 group by topic,user',conn)

CPU times: user 2.67 s, sys: 23.1 ms, total: 2.69 s
Wall time: 8.77 s


###Make usernames lower case

In [5]:
postsDf['user']=postsDf['user'].apply(unicode.lower)

In [6]:
postsDf.head()

,user,topic,count(*)
0,0393,Adverts,1
1,100ksalescom,Adverts,3
2,1nigerian,Adverts,1
3,247hostwahala,Adverts,1
4,2marto,Adverts,1


In [7]:
usersDf.head()

,userName,gender,posts
0,lildonne,m,71
1,dejt4u,m,4
2,macpetrus,m,567
3,disloman,m,943
4,idrismusty97,m,97


In [8]:
usersDf['user']=usersDf['userName']
del usersDf['userName']

###Merge posts and users based on username

In [9]:
joined=pd.merge(postsDf,usersDf,on='user')

In [10]:
joined.shape

(88138, 5)

In [11]:
usersDf[usersDf['user']=='lildonne']

,gender,posts,user
0,m,71,lildonne


In [12]:
postsDf[postsDf['user']=='lildonne']

,user,topic,count(*)
10202,lildonne,Car Talk,1
15974,lildonne,Celebrity Crimes,1
21717,lildonne,Education,1
60959,lildonne,Politics,2


###Get male and female users

In [13]:
maleJoined=joined[joined['gender']==u' m']
femaleJoined=joined[joined['gender']==u' f']

####Male and female users are unbalanced

In [14]:
print maleJoined.shape
print femaleJoined.shape
print '%1.2f' % (maleJoined.shape[0]/float(femaleJoined.shape[0]))

(42400, 5)
(6261, 5)
6.77


In [15]:
#maleMatrix=pd.pivot_table(~maleJoined[maleJoined['user'].isin(['_MISSING_'])],index=['user'],columns=['topic'],fill_value=0)['posts']
maleMatrix=pd.pivot_table(maleJoined,index=['user'],columns=['topic'],fill_value=0)['posts']
femaleMatrix=pd.pivot_table(femaleJoined,index=['user'],columns=['topic'],fill_value=0)['posts']

In [16]:
maleMatrix.head()

topic,Adverts,Agriculture,"Art, Graphics & Video",Autos,Business,Business To Business,Car Talk,Career,Celebrities,Celebrity Crimes,...,Software/Programmer Market,Sports,TV/Movies,Technology Market,Travel,Travel Ads,Video Games And Gadgets For Sale,Web Market,Webmasters,up Zone
user,,,,,,,,,,,,,,,,,,,,,
01emek,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01karma,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
04mike,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
080web,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,80,0
0955eb027,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
femaleMatrix.head()

topic,Adverts,Agriculture,"Art, Graphics & Video",Autos,Business,Business To Business,Car Talk,Career,Celebrities,Celebrity Crimes,...,Sports,TV/Movies,Technology Market,Top Pages,Travel,Travel Ads,Video Games And Gadgets For Sale,Web Market,Webmasters,up Zone
user,,,,,,,,,,,,,,,,,,,,,
1mindchrisr,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1ndidi2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
222rayo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2swt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
310mbdotcom,0,0,0,0,0,0,0,0,0,0,...,0,310,0,0,0,0,0,0,0,0


###Sum over all topics for each user to get total number of posts

In [18]:
maleMatrix['posts']=maleMatrix.sum(axis=1)
femaleMatrix['posts']=femaleMatrix.sum(axis=1)

### (opt) Filter by total number of posts

In [689]:
nCutOff=3

print maleMatrix.shape
maleMatrix=maleMatrix[maleMatrix['posts']>nCutOff]
print maleMatrix.shape
print '----------'
print femaleMatrix.shape
femaleMatrix=femaleMatrix[femaleMatrix['posts']>nCutOff]
print femaleMatrix.shape

(11977, 64)
(11977, 64)
----------
(2079, 63)
(2079, 63)


###Add gender labels

In [19]:
maleMatrix['gender']=u'm'
femaleMatrix['gender']=u'f'

###Some topics are exclusive to a gender

In [20]:
print maleMatrix.shape
print femaleMatrix.shape
print 'Inbalance %.3f' % (maleMatrix.shape[0]/float(femaleMatrix.shape[0]))

(16238, 64)
(2922, 63)
Inbalance 5.557


In [692]:
for n in np.nonzero(sum(np.isnan(maleFemaleData)))[0]:
    print maleFemale.columns.values[n],sum(np.isnan(maleFemaleData))[n]

###Combine both genders

In [97]:
if False:
    maleFemale=pd.concat([maleMatrix,femaleMatrix])
elif True: # Only if certain number of posts
    nCutOff=15
    maleFemale=pd.concat([maleMatrix[maleMatrix.posts>nCutOff],femaleMatrix[femaleMatrix.posts>nCutOff]])
else: # Balance artificially through undersampling
    maleFemale=pd.concat([maleMatrix.sample(femaleMatrix.shape[0]),femaleMatrix])    
print maleFemale.shape

(14678, 65)


###So set NaN's to zeros

In [98]:
maleFemale.fillna(0,inplace=True)

###Get data (ignore target labels)

In [91]:
print list(maleFemale.columns.values).index('gender')
print list(maleFemale.columns.values).index('posts')

62
63


In [46]:
maleFemaleData=maleFemale.iloc[:,0:62].as_matrix()
# Get all topic counts as features (excludes counts)

In [99]:
maleFemaleData=maleFemale.icol(range(62)+[63]).as_matrix()
# Get all topic counts as features (includes counts)

In [697]:
#maleFemaleData=maleFemale['posts'].as_matrix()
# Get just number of posts as features

###(opt) binarise

In [698]:
from sklearn.preprocessing import Binarizer

In [699]:
binarizer=Binarizer().fit(maleFemaleData)

In [700]:
maleFemaleData=binarizer.transform(maleFemaleData)

###(opt) Normalise

In [83]:
from sklearn.preprocessing import normalize

In [106]:
maleFemaleData=normalize(maleFemaleData, norm='l2')

###Get labels

In [102]:
maleFemaleLabels=maleFemale.gender.values

###Try to classify

###Decision Tree

In [107]:
from sklearn.cross_validation import cross_val_score
from sklearn import tree
clf=tree.DecisionTreeClassifier()

In [108]:
clf.fit(maleFemaleData,maleFemaleLabels)

DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            random_state=None, splitter='best')

In [109]:
cross_val_score(clf,maleFemaleData,maleFemaleLabels,cv=5)

array([ 0.81818182,  0.82595368,  0.82657581,  0.81942078,  0.83577513])

In [37]:
print len(clf.feature_importances_)
print len(maleFemale.columns)

62
65


In [70]:
for label,value in zip(maleFemale.columns,clf.feature_importances_):
    print '%s - %.3f' % (label,value)

Adverts - 0.010
Agriculture - 0.006
Art, Graphics & Video - 0.004
Autos - 0.022
Business - 0.028
Business To Business - 0.005
Car Talk - 0.008
Career - 0.014
Celebrities - 0.024
Celebrity Crimes - 0.005
Certification And Training Adverts - 0.001
Computer Market - 0.002
Computers - 0.011
Culture - 0.006
Education - 0.029
Educational Services - 0.002
Entertainment - 0.000
European Football (EPL, UEFA, La Liga) - 0.004
Events - 0.011
Family - 0.033
Fashion - 0.034
Fashion/Clothing Market - 0.006
Food - 0.010
Foreign Affairs - 0.008
Forum Games - 0.001
Gaming - 0.004
Graphics/Video Market - 0.000
Health - 0.023
Investment - 0.004
Investment Ads - 0.000
Islam for Muslims - 0.006
Jobs/Vacancies - 0.034
Jokes Etc - 0.022
Literature - 0.017
Literature/Writing Ads - 0.001
Music Business - 0.001
Music/Radio - 0.011
NYSC - 0.009
Nairaland / General - 0.038
Nairaland Ads - 0.000
Pets - 0.007
Phone/Internet Market - 0.007
Phones - 0.031
Poems For Review - 0.003
Politics - 0.068
Programming - 0.006


###Naive Bayes

In [732]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [733]:
np.nonzero(sum(np.isnan(maleFemaleData)))

(array([], dtype=int64),)

In [734]:
for n in np.nonzero(sum(np.isnan(maleFemaleData)))[0]:
    print maleFemale.columns.values[n]

In [735]:
y_pred = gnb.fit(maleFemaleData,maleFemaleLabels).predict(maleFemaleData)

In [736]:
print("Number of mislabeled points out of a total %d points : %d"
      % (maleFemaleData.shape[0],(maleFemaleLabels != y_pred).sum()))
print '%.3f' % ((maleFemaleLabels != y_pred).sum()/float(maleFemaleData.shape[0]))

Number of mislabeled points out of a total 4158 points : 1653
0.398


###SVM

In [737]:
from sklearn import svm

In [738]:
clf = svm.SVC()

In [739]:
%time clf.fit(maleFemaleData,maleFemaleLabels)

CPU times: user 1.77 s, sys: 0 ns, total: 1.77 s
Wall time: 1.77 s


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [740]:
print sum(clf.predict(maleFemaleData)==maleFemaleLabels)
print maleFemaleLabels.shape
print 'Accuracy => %.3f' % (sum(clf.predict(maleFemaleData)==maleFemaleLabels)/float(maleFemaleLabels.shape[0]))

2822
(4158,)
Accuracy => 0.679


###Decision Tree

In [741]:
from sklearn import tree

In [742]:
clf = tree.DecisionTreeClassifier()

In [743]:
%time clf.fit(maleFemaleData,maleFemaleLabels)

CPU times: user 85.3 ms, sys: 0 ns, total: 85.3 ms
Wall time: 84.7 ms


DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            random_state=None, splitter='best')

In [744]:
print sum(clf.predict(maleFemaleData)==maleFemaleLabels)
print maleFemaleLabels.shape
print '%.3f' % (sum(clf.predict(maleFemaleData)==maleFemaleLabels)/float(maleFemaleLabels.shape[0]))

3386
(4158,)
0.814


In [745]:
importances=collections.Counter({f:i for f,i in zip(maleFemale.columns.values,clf.feature_importances_)})

In [746]:
importances.most_common(10)

[(u'Politics', 0.075384448891034447),
 (u'Family', 0.056208706197512129),
 (u'Fashion', 0.05289738260950598),
 (u'Romance', 0.042456516373629544),
 (u'Sports', 0.041678381395890716),
 (u'Celebrities', 0.038840393199254204),
 (u'Nairaland / General', 0.038728365649402872),
 (u'Autos', 0.038116851657633517),
 (u'Health', 0.03581939749956705),
 (u'Phones', 0.032049506544273243)]

###Train/Test Split

In [747]:
from sklearn.cross_validation import train_test_split

In [748]:
dataTrain,dataTest,labelTrain,labelTest=train_test_split(maleFemaleData,maleFemaleLabels,test_size=0.2)
print dataTrain.shape,dataTest.shape
print labelTrain.shape,labelTest.shape

(3326, 62) (832, 62)
(3326,) (832,)


In [749]:
clf = tree.DecisionTreeClassifier()
#clf = svm.SVC()

In [750]:
clf.fit(dataTrain,labelTrain)

DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            random_state=None, splitter='best')

In [751]:
print sum(clf.predict(dataTest)==labelTest)
print '%0.2f' % (sum(clf.predict(dataTest)==labelTest)/float(labelTest.shape[0]))

533
0.64
